In [12]:
import numpy as np

In [13]:
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevanz score < k')
    return np.mean(r)

In [14]:
def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

In [15]:
def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

In [16]:
test1 = [[1, 1, 0, 1, 0], [0, 0, 1, 1, 1]] # BA Beispiel
test2 = [[1, 1, 1, 1, 1], [0, 0, 0, 0, 0]]
test3 = [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]
test4 = [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]


print(mean_average_precision(test1))
print(mean_average_precision(test2))
print(mean_average_precision(test3))
print(mean_average_precision(test4))

0.6972222222222222
0.5
1.0
0.0


In [17]:
from flask import Flask, json, request

In [18]:

def json_response(payload, status=200):
    return (json.dumps(payload), status, {'content-type': 'application/json'})

In [19]:
jsonTest = {'0': [{'score': 60.9176157115223, 'rating': 1, 'ranking': 0}, {'score': 57.875198563054134, 'rating': 1, 'ranking': 1}, {'score': 57.875198563054134, 'rating': 0, 'ranking': 2}, {'score': 57.875198563054134, 'rating': 0, 'ranking': 3}, {'score': 54.2979302092234, 'rating': 1, 'ranking': 6}, {'score': 57.875198563054134, 'rating': 1, 'ranking': 4}, {'score': 54.469250578633385, 'rating': 1, 'ranking': 5}, {'score': 50.89198222480265, 'rating': 1, 'ranking': 7}, {'score': 50.89198222480265, 'rating': 0, 'ranking': 8}, {'score': 50.89198222480265, 'rating': 1, 'ranking': 9}], '1': [{'score': 74.12525988109097, 'rating': 1, 'ranking': 6}, {'score': 87.23094749123473, 'rating': 1, 'ranking': 0}, {'score': 81.03058575184154, 'rating': 0, 'ranking': 1}, {'score': 80.32562162048416, 'rating': 1, 'ranking': 2}, {'score': 74.12525988109097, 'rating': 0, 'ranking': 3}, {'score': 74.12525988109097, 'rating': 0, 'ranking': 4}, {'score': 74.12525988109097, 'rating': 1, 'ranking': 5}, {'score': 74.12525988109097, 'rating': 1, 'ranking': 7}, {'score': 74.12525988109097, 'rating': 1, 'ranking': 8}, {'score': 74.12525988109097, 'rating': 1, 'ranking': 9}], '2': [{'score': 102.30070084356836, 'rating':
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        1, 'ranking': 0}, {'score': 102.30070084356836, 'rating': 0, 'ranking': 1}, {'score':
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     102.30070084356836, 'rating': 0, 'ranking': 2}, {'score': 102.30070084356836, 'rating': 1, 'ranking': 3}, {'score': 102.30070084356836, 'rating': 1, 'ranking': 4}, {'score': 102.30070084356836, 'rating': 1, 'ranking': 5}, {'score': 102.30070084356836, 'rating': 1, 'ranking': 7}, {'score': 102.30070084356836, 'rating': 0, 'ranking': 6}, {'score': 102.30070084356836, 'rating': 0, 'ranking': 8}, {'score': 102.30070084356836, 'rating': 1, 'ranking': 9}], '3': [{'score': 38.76666340151514, 'rating': 1, 'ranking': 0}, {'score': 38.76666340151514, 'rating': 0, 'ranking': 1}, {'score': 38.76666340151514, 'rating': 1, 'ranking': 2}, {'score': 38.76666340151514, 'rating': 1, 'ranking': 3}, {'score': 38.76666340151514, 'rating': 1, 'ranking': 4}]}


In [20]:
print(jsonTest)

{'0': [{'score': 60.9176157115223, 'rating': 1, 'ranking': 0}, {'score': 57.875198563054134, 'rating': 1, 'ranking': 1}, {'score': 57.875198563054134, 'rating': 0, 'ranking': 2}, {'score': 57.875198563054134, 'rating': 0, 'ranking': 3}, {'score': 54.2979302092234, 'rating': 1, 'ranking': 6}, {'score': 57.875198563054134, 'rating': 1, 'ranking': 4}, {'score': 54.469250578633385, 'rating': 1, 'ranking': 5}, {'score': 50.89198222480265, 'rating': 1, 'ranking': 7}, {'score': 50.89198222480265, 'rating': 0, 'ranking': 8}, {'score': 50.89198222480265, 'rating': 1, 'ranking': 9}], '1': [{'score': 74.12525988109097, 'rating': 1, 'ranking': 6}, {'score': 87.23094749123473, 'rating': 1, 'ranking': 0}, {'score': 81.03058575184154, 'rating': 0, 'ranking': 1}, {'score': 80.32562162048416, 'rating': 1, 'ranking': 2}, {'score': 74.12525988109097, 'rating': 0, 'ranking': 3}, {'score': 74.12525988109097, 'rating': 0, 'ranking': 4}, {'score': 74.12525988109097, 'rating': 1, 'ranking': 5}, {'score': 74.1

In [21]:
def calc_map(data):
    all_ratings = []
    for query in data:
        rating_arr = data[query]
        sorted_arr = sorted(rating_arr, key=lambda k: k['ranking'])
        query_ratings = []
        for res in sorted_arr:
            query_ratings.append(res['rating'])
        all_ratings.append(query_ratings)
    
    return mean_average_precision(all_ratings)

calc_map(jsonTest)

0.7302437641723356

In [22]:
def calc_avg_tfidf(data):
    avg_tfidf = []
    rating_sum = 0
    for query in data:
        rating_arr = data[query]
        score_sum = 0
        for rating in rating_arr:
            score = rating['score']
            score_sum = float(score_sum) + float(score)
        avg_score = round(score_sum/len(rating_arr), 2)
        avg_tfidf.append(avg_score)

    return avg_tfidf


print(calc_avg_tfidf(jsonTest))


[55.39, 76.75, 102.3, 38.77]


In [23]:
def calc_highest_tfidf(data):
    highest_arr = []
    for query in data:
        rating_arr = data[query]
        sorted_arr = sorted(rating_arr, key=lambda k: k['ranking'])
        highest_arr.append(sorted_arr[0]['score'])
    
    return highest_arr

print(calc_highest_tfidf(jsonTest))

[60.9176157115223, 87.23094749123473, 102.30070084356836, 38.76666340151514]


In [74]:
import numpy as np

def cum_gain(relevance):
    return np.asarray(relevance).sum()


def dcg(relevance):
    rel = np.asarray(relevance)
    p = len(rel)

    log2i = np.log2(np.asarray(range(1, p + 1)) + 1)
    return ((np.power(2, rel) - 1) / log2i).sum()



def idcg(relevance):
    rel = np.asarray(relevance).copy()
    rel.sort()
    return dcg(rel[::-1])


def ndcg(relevance, nranks = 10):
    rel = np.asarray(relevance)
    pad = max(0, nranks - len(rel))

    rel = np.pad(rel, (0, pad), 'constant')

    rel = rel[0:min(nranks, len(rel))]

    ideal_dcg = idcg(rel)
    if ideal_dcg == 0:
        return 0.0

    return dcg(rel) / ideal_dcg

r = [3, 1, 1, 1, 1, 1, 1, 1, 0, 0]
ndcg(r, 10)

1.0

In [64]:
def calc_ndcg(data):
    ndcg_score = {}
    for query in data:
        rating_arr = data[query]
        sorted_arr = sorted(rating_arr, key=lambda k: k['ranking'])
        query_ratings = []
        for res in sorted_arr:
            query_ratings.append(res['rating'])
        ndcg_value = ndcg(query_ratings)
        ndcg_score[query] = ndcg_value
    return ndcg_score



{'0': 0.9103499485143787, '1': 0.8507697982583751, '2': 0.840710074262534, '3': 0.9047172294870751}


In [56]:
def calc_avg_tfidf(data):
    avg_tfidf = {}
    for query in data:
        rating_arr = data[query]
        score_sum = 0
        for rating in rating_arr:
            score = rating['score']
            score_sum = float(score_sum) + float(score)
        avg_score = round(score_sum/len(rating_arr), 2)
        avg_tfidf[query] = avg_score

    return avg_tfidf

calc_avg_tfidf(jsonTest)

{'0': 55.39, '1': 76.75, '2': 102.3, '3': 38.77}

In [13]:
import uuid
uuid.uuid4().hex

'f42bd18d427a433c9b51270bf607a1b5'

In [15]:
def calc_precision(data):
    precision = []
    for query in data:
        rating_arr = data[query]
        good_count = 0
        for rating in rating_arr:
            if(rating['rating'] == 1):
                good_count = good_count + 1
        precision.append((good_count/len(rating_arr)))
    return precision

print(calc_precision(jsonTest))

[0.7, 0.7, 0.6, 0.8]


In [32]:
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def calc_precision_at_k(data, k):
    all_ratings = []
    for query in data:
        rating_arr = data[query]
        sorted_arr = sorted(rating_arr, key=lambda k: k['ranking'])
        query_ratings = []
        for res in sorted_arr:
            query_ratings.append(res['rating'])
        all_ratings.append(precision_at_k(query_ratings, k))
    
    return all_ratings
    
print(calc_precision_at_k(jsonTest, 5))

[0.6, 0.4, 0.6, 0.8]


In [68]:

file = r"C:\\Users\\lucas\\Documents\\Github\\BA_Search_Engine\\backend\\repository\\data\\metrics.json"
def load_metrics():
    
    with open(file, 'r') as f:
        data = json.load(f)
    return data

data = load_metrics()


In [51]:
import pandas as pd
import json

file = r"C:\\Users\\lucas\\Documents\\Github\\BA_Search_Engine\\backend\\repository\\data\\metrics.json"
with open(file, encoding="utf8") as json_file:
    data = json.load(json_file)

In [52]:
# for user in data:
#     print(user)
#     search_type = data[user]
#     mlMetrics = data[user]['ml']['metrics']
#     ftMetrics = data[user]['ft']['metrics']
#     # print(mlMetrics)
#     for metric in ftMetrics:
#         if metric != 'map':
#             for queryIndex in ftMetrics[metric]:
#                 value =  ftMetrics[metric][queryIndex]
#                 print(value)
# len(data)

In [56]:
import xlsxwriter

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('metrics.xlsx')
worksheet = workbook.add_worksheet()

header_format = workbook.add_format({
    'bold': 1,
    'align': 'center',
})

subHeader_format = workbook.add_format({
    'bold': 0,
    'align': 'center',
})

# Start from the first cell. Rows and columns are zero indexed.
row = 0
col = 0

user_count = len(data)
query_count = 10
count = 1
for user in data:
    user_row = count
    worksheet.merge_range(f'B{user_row}:K{user_row}', user, header_format)
    worksheet.merge_range(f'B{user_row+1}:F{user_row+1}', 'ML-Searchengine', subHeader_format)
    worksheet.write(f'B{user_row+2}', 'Avg. TF-IDF')
    worksheet.write(f'C{user_row+2}', 'Top TF-IDF')
    worksheet.write(f'D{user_row+2}', 'nDCG')
    worksheet.write(f'E{user_row+2}', 'Precision')

    worksheet.merge_range(f'G{user_row+1}:K{user_row+1}', 'Fulltext-Searchengine', subHeader_format)
    worksheet.write(f'G{user_row+2}', 'Avg. TF-IDF')
    worksheet.write(f'H{user_row+2}', 'Top TF-IDF')
    worksheet.write(f'I{user_row+2}', 'nDCG')
    worksheet.write(f'J{user_row+2}', 'Precision')

    mlMetrics = data[user]['ml']['metrics']
    ftMetrics = data[user]['ft']['metrics']

    for i in range(query_count):
        worksheet.write(user_row+2+i, 0, f'Query {i+1}')

    m_row = user_row + 2
    m_col = 1
    for metric in mlMetrics:
        if metric != 'map':
            for queryIndex in mlMetrics[metric]:
                value =  mlMetrics[metric][queryIndex]
                worksheet.write(m_row+int(queryIndex), m_col, value)
            m_col += 1
            m_row = user_row + 2
    
    m_col = len(mlMetrics) + 1 # anzahl an metriken + 1 weil die erste Spalte für Query Bezeichnung genutzt wird
    for metric in ftMetrics:
        if metric != 'map':
            for queryIndex in ftMetrics[metric]:
                value =  ftMetrics[metric][queryIndex]
                worksheet.write(m_row+int(queryIndex), m_col, value)
            m_col += 1
            m_row = user_row + 2
        

        
    worksheet.write(query_count + count + 2, 0, 'MAP')
    worksheet.write(query_count + count + 2, 1, mlMetrics['map'])
    worksheet.write(query_count + count + 2, len(mlMetrics) + 1, ftMetrics['map'])



    count  = count + query_count + 4 # 4 weil 3 header über den queries + map Zeile

workbook.close()